In [2]:
import json
import requests
import os
import pandas as pd
from urllib.parse import quote

In [3]:
file = open('../key.json', 'r')
API_KEY = json.load(file)[0]['OMDB_API']

In [63]:
# movie_dict = pd.read_csv("movies.csv").drop("Unnamed: 0", axis=1)
# movie_dict.to_csv("movies.csv", index=False)

In [119]:
movie_dict = pd.read_csv("movies.csv").to_dict(orient="records")
movies = [i['title'].lower().replace("º", "°").replace("’", "'") for i in movie_dict] #movie list

In [120]:
scrapped = pd.read_csv("movie_info.csv")
scrapped.drop_duplicates(inplace=True)
scrapped_dict = scrapped.to_dict(orient="records")
scrapped_movies = [i['Title'].lower() for i in scrapped_dict]

In [121]:
# "Alexander and the Terrible, Horrible, No Good, …".replace(" …", "")
# "Alexander and the Terrible, Horrible, No Good, Very Bad Day".find("Alexander and the Terrible, Horrible, No Good,")

# l = ["done", "Alexander and the Terrible, Horrible, No Good, Very Bad Day"]
# l[1].endswith(" …")
# scrapped = scrapped.drop("Unnamed: 0.1", axis=1)
# scrapped = scrapped.drop("Unnamed: 0", axis=1)

# scrapped = scrapped.drop(272)
# scrapped = scrapped.drop(743)
# scrapped = scrapped.drop(776)
# scrapped.to_csv("movie_info.csv", index=False)

"no se aceptan devoluciones 2: el regreso de loreto peralta".find("no se aceptan devoluciones")

0

In [122]:
for index, i in enumerate(movies):
    if i.endswith("…"):
        movies[index] = i.replace("…", "").strip()
        # print(i)

In [123]:
movies = list(set(movies))
scrapped_movies = list(set(scrapped_movies))

In [124]:
leftover = []
for movie in movies:
    if movie not in scrapped_movies:
        leftover.append(movie)

In [125]:
df1 = pd.DataFrame({'movies': movies})
df2 = pd.DataFrame({'leftover': leftover})
df3 = pd.DataFrame({'scrapped':scrapped_movies})

result_df = pd.merge(df1, df2, left_on='movies', right_on='leftover', how='outer')

result_df = pd.merge(result_df, df3, left_on='movies', right_on='scrapped', how='outer')

result_df.to_csv("output.csv")


In [126]:
len(movies), len(scrapped_movies), len(leftover) , (len(scrapped_movies)+len(leftover))

(2006, 737, 1271, 2008)

In [136]:
new_list = []
count = 0
not_added = 0

for m in leftover:
    formatted = quote(m)
    url = f"http://www.omdbapi.com/?t={formatted}&apikey=2c60d316"
    response = requests.get(url)
    # print(url)
    if response.status_code == 200:
        data = json.loads(response.text)
        if data['Response'] != 'False':
            new_list.append(data)
            print(data['Title'], m)
            count += 1
        else:
            not_added += 1
    else:
        print(f"Status code: {response.status_code}")
        break

The Star the star
Hostiles hostiles
Columbus columbus
The Veil the veil
Glass glass
God's Not Dead 2 god's not dead 2
The Florida Project the florida project
London Fields london fields
Logan Lucky logan lucky
Dangal dangal
Jungle Cruise jungle cruise
The Last Duel the last duel
Show Dogs show dogs
The Menu the menu
Godzilla: King of the Monsters godzilla: king of the monsters
Stuber stuber
Hamsterit hamsterit
The Other Side of the Door the other side of the door
Batman v Superman: Dawn of Justice batman v superman: dawn of justice
West Side Story west side story
Vuosisadan häät vuosisadan häät
Profile profile
Morbius morbius
Can You Ever Forgive Me? can you ever forgive me?
Ferrari ferrari
The Disaster Artist the disaster artist
Toy Story 4 toy story 4
Alien: Covenant alien: covenant
An Inconvenient Sequel: Truth to Power an inconvenient sequel
Queen & Slim queen & slim
12 Strong 12 strong
Bodom bodom
The Kid the kid
10 Cloverfield Lane 10 cloverfield lane
Greater greater
Going in Sty

ConnectionError: HTTPConnectionPool(host='www.omdbapi.com', port=80): Max retries exceeded with url: /?t=the%20nut%20job%202%3A%20nutty%20by%20nature&apikey=2c60d316 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fe94255d8a0>: Failed to establish a new connection: [Errno 101] Network is unreachable'))

In [143]:
new_scrapped = pd.DataFrame(new_list)
len(new_scrapped), len(scrapped), new_scrapped.columns, scrapped.columns

(200,
 803,
 Index(['Title', 'Year', 'Rated', 'Released', 'Runtime', 'Genre', 'Director',
        'Writer', 'Actors', 'Plot', 'Language', 'Country', 'Awards', 'Poster',
        'Ratings', 'Metascore', 'imdbRating', 'imdbVotes', 'imdbID', 'Type',
        'DVD', 'BoxOffice', 'Production', 'Website', 'Response'],
       dtype='object'),
 Index(['Title', 'Year', 'Rated', 'Released', 'Runtime', 'Genre', 'Director',
        'Writer', 'Actors', 'Plot', 'Language', 'Country', 'Awards', 'Poster',
        'Ratings', 'Metascore', 'imdbRating', 'imdbVotes', 'imdbID', 'Type',
        'DVD', 'BoxOffice', 'Production', 'Website', 'totalSeasons',
        'Response'],
       dtype='object'))

In [147]:
merged_new = pd.concat([scrapped, new_scrapped], ignore_index = True).reset_index()

In [148]:
merged_new.to_csv('new_merged_scrapped.csv', index=False)